In [ ]:
#論文通り0.0001unitだけ進める

import numpy as np
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

# 定数
F = 8.0
N = 40
sample = 50
dt = 0.05
epsilon = 1e-4
tend_attract = 20

def L96(t, x, F):
    dxdt = np.zeros(N)
    for i in range(2, N-1):
        dxdt[i] = (x[i+1] - x[i-2]) * x[i-1] - x[i] + F
    dxdt[0] = (x[1] - x[N-2]) * x[N-1] - x[0] + F
    dxdt[1] = (x[2] - x[N-1]) * x[0] - x[1] + F
    dxdt[N-1] = (x[0] - x[N-3]) * x[N-2] - x[N-1] + F
    return dxdt

# 初期条件
X = F + np.random.randn(N)

# 時間スパン設定
tspan = (0.0, 100)
t_eval = np.linspace(tspan[0], tspan[1], int(tspan[1] / dt) + 1)

# ODEを解く
sol = solve_ivp(L96, tspan, X, args=(F,), method='RK45', t_eval=t_eval)

# 解の補間関数を作成
sol_interp = interp1d(sol.t, sol.y, kind='cubic', axis=1, fill_value="extrapolate")

# 誤差評価用配列
error = np.zeros((sample, int(tend_attract / dt) + 1))
t_perturb = np.linspace(250, 500, int(250 / epsilon) + 1)

X_attract = sol.y[:, int(len(t_eval) / 2) + int(1 / dt) * i] 
#少しずらす
X_sol_interp = sol_interp(t_perturb)

for i in range(sample):
    X_attract_perturb = X_sol_interp[:, int(len(t_perturb) / 2) + int(1 / dt) * 5000 * i + 1] 
    
    sol_attract = solve_ivp(L96, (0, tend_attract), X_attract, args=(F,), method='RK45', dense_output=True)
    sol_attract_perturb = solve_ivp(L96, (0, tend_attract), X_attract_perturb, args=(F,), method='RK45', dense_output=True)
    #誤差の時間発展間隔
    tspan_attract = np.linspace(0.0, tend_attract, int(tend_attract * int(1 / dt) + 1))
    attract = sol_attract.sol(tspan_attract)#補間関数('sol')
    attract_perturb = sol_attract_perturb.sol(tspan_attract)
    # 時間ごとのノルムの差
    for j in range(len(tspan_attract)):
        error[i, j] = np.linalg.norm(attract[:, j] - attract_perturb[:, j])
        
# エラーの平均を計算
error_mean = np.mean(error, axis=0)
#誤差評価用の配列(リアプノフ指数)
error_lyapunov = np.zeros(len(error_mean))
for i in range(len(error_mean)):
    error_lyapunov[i] = np.log(abs(error_mean[i] / error_mean[0])) / (i + 1)

# 結果のプロット
plt.figure(figsize=(10, 6))
plt.plot(tspan_attract, error_lyapunov, label='Mean Error')
plt.xlabel('Time(steps)')
plt.ylabel('Lyapunov Exponent')
plt.title('Error Development over Time')
plt.grid()
plt.show()
   